# S41 Algorithm Analysis

Analysis of the S41 team's solution to the ROADEF 2012 Machine Reassignment Problem.

This notebook analyzes the process reassignments and algorithm behavior.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import ast

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


In [ ]:
# Load the tracking data
sol1_data = pd.read_csv('../process_reassignments_sol1.csv')
sol2_data = pd.read_csv('../process_reassignments_sol2.csv')

print(f"Solution 1: {len(sol1_data)} reassignments")
print(f"Solution 2: {len(sol2_data)} reassignments")
print("\nFirst few rows of Solution 1:")
sol1_data.head()


In [ ]:
# Basic statistics about the reassignments
def analyze_reassignments(data, name):
    print(f"\n=== {name} Analysis ===")
    print(f"Total reassignments: {len(data)}")
    print(f"Unique processes moved: {data['ProcessID'].nunique()}")
    print(f"Unique source machines: {data['SourceMachine'].nunique()}")
    print(f"Unique destination machines: {data['DestMachine'].nunique()}")
    print(f"Final solution cost: {data['SolutionCost'].iloc[-1]}")
    print(f"Cost improvement: {data['SolutionCost'].iloc[0] - data['SolutionCost'].iloc[-1]}")
    
    return data

sol1_analysis = analyze_reassignments(sol1_data, "Solution 1")
sol2_analysis = analyze_reassignments(sol2_data, "Solution 2")


In [ ]:
# Plot cost evolution over time
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Solution 1 costs
axes[0, 0].plot(sol1_data['MoveNum'], sol1_data['SolutionCost'], alpha=0.7)
axes[0, 0].set_title('Solution 1: Total Cost Evolution')
axes[0, 0].set_xlabel('Move Number')
axes[0, 0].set_ylabel('Total Cost')

axes[0, 1].plot(sol1_data['MoveNum'], sol1_data['LoadCost'], alpha=0.7, label='Load Cost')
axes[0, 1].plot(sol1_data['MoveNum'], sol1_data['BalanceCost'], alpha=0.7, label='Balance Cost')
axes[0, 1].set_title('Solution 1: Component Costs')
axes[0, 1].set_xlabel('Move Number')
axes[0, 1].set_ylabel('Cost')
axes[0, 1].legend()

# Solution 2 costs
axes[1, 0].plot(sol2_data['MoveNum'], sol2_data['SolutionCost'], alpha=0.7, color='orange')
axes[1, 0].set_title('Solution 2: Total Cost Evolution')
axes[1, 0].set_xlabel('Move Number')
axes[1, 0].set_ylabel('Total Cost')

axes[1, 1].plot(sol2_data['MoveNum'], sol2_data['LoadCost'], alpha=0.7, label='Load Cost')
axes[1, 1].plot(sol2_data['MoveNum'], sol2_data['BalanceCost'], alpha=0.7, label='Balance Cost')
axes[1, 1].set_title('Solution 2: Component Costs')
axes[1, 1].set_xlabel('Move Number')
axes[1, 1].set_ylabel('Cost')
axes[1, 1].legend()

plt.tight_layout()
plt.show()


In [ ]:
# Compare the two solutions
plt.figure(figsize=(12, 6))

# Resample data for better visualization (every 100th point)
sol1_sample = sol1_data[::100] if len(sol1_data) > 1000 else sol1_data
sol2_sample = sol2_data[::100] if len(sol2_data) > 1000 else sol2_data

plt.plot(sol1_sample['MoveNum'], sol1_sample['SolutionCost'], label='Solution 1', alpha=0.8)
plt.plot(sol2_sample['MoveNum'], sol2_sample['SolutionCost'], label='Solution 2', alpha=0.8)
plt.title('Comparison of Cost Evolution Between Solutions')
plt.xlabel('Move Number')
plt.ylabel('Solution Cost')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Final costs:")
print(f"Solution 1: {sol1_data['SolutionCost'].iloc[-1]}")
print(f"Solution 2: {sol2_data['SolutionCost'].iloc[-1]}")
print(f"Better solution: {'Solution 1' if sol1_data['SolutionCost'].iloc[-1] < sol2_data['SolutionCost'].iloc[-1] else 'Solution 2'}")


In [ ]:
# Analyze process mobility patterns
def analyze_process_mobility(data, name):
    print(f"\n=== {name} Process Mobility ===")
    
    # Count how many times each process was moved
    process_moves = data['ProcessID'].value_counts()
    
    print(f"Most frequently moved processes:")
    print(process_moves.head(10))
    
    print(f"\nMovement distribution:")
    print(f"Processes moved once: {sum(process_moves == 1)}")
    print(f"Processes moved 2-5 times: {sum((process_moves >= 2) & (process_moves <= 5))}")
    print(f"Processes moved >5 times: {sum(process_moves > 5)}")
    print(f"Max moves for single process: {process_moves.max()}")
    
    return process_moves

sol1_mobility = analyze_process_mobility(sol1_data, "Solution 1")
sol2_mobility = analyze_process_mobility(sol2_data, "Solution 2")


In [ ]:
# Visualize process mobility distributions
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Solution 1
axes[0].hist(sol1_mobility.values, bins=50, alpha=0.7, edgecolor='black')
axes[0].set_title('Solution 1: Distribution of Moves per Process')
axes[0].set_xlabel('Number of Moves')
axes[0].set_ylabel('Number of Processes')
axes[0].set_yscale('log')

# Solution 2
axes[1].hist(sol2_mobility.values, bins=50, alpha=0.7, edgecolor='black', color='orange')
axes[1].set_title('Solution 2: Distribution of Moves per Process')
axes[1].set_xlabel('Number of Moves')
axes[1].set_ylabel('Number of Processes')
axes[1].set_yscale('log')

plt.tight_layout()
plt.show()


In [ ]:
# Analyze machine utilization patterns
def analyze_machine_patterns(data, name):
    print(f"\n=== {name} Machine Patterns ===")
    
    source_counts = data['SourceMachine'].value_counts()
    dest_counts = data['DestMachine'].value_counts()
    
    print(f"Most active source machines (most processes moved from):")
    print(source_counts.head(10))
    
    print(f"\nMost active destination machines (most processes moved to):")
    print(dest_counts.head(10))
    
    return source_counts, dest_counts

sol1_sources, sol1_dests = analyze_machine_patterns(sol1_data, "Solution 1")
sol2_sources, sol2_dests = analyze_machine_patterns(sol2_data, "Solution 2")


In [ ]:
# Create a summary report
print("\n" + "="*50)
print("SUMMARY REPORT: S41 Algorithm Analysis")
print("="*50)

print(f"\nDataset: A1_2 (based on file paths)")
print(f"Algorithm: S41 Team Solution (C++)")
print(f"Run time: ~10 seconds")

print(f"\nSolution 1 Results:")
print(f"  - Total moves: {len(sol1_data):,}")
print(f"  - Unique processes moved: {sol1_data['ProcessID'].nunique():,}")
print(f"  - Final cost: {sol1_data['SolutionCost'].iloc[-1]:,}")
print(f"  - Cost reduction: {sol1_data['SolutionCost'].iloc[0] - sol1_data['SolutionCost'].iloc[-1]:,}")

print(f"\nSolution 2 Results:")
print(f"  - Total moves: {len(sol2_data):,}")
print(f"  - Unique processes moved: {sol2_data['ProcessID'].nunique():,}")
print(f"  - Final cost: {sol2_data['SolutionCost'].iloc[-1]:,}")
print(f"  - Cost reduction: {sol2_data['SolutionCost'].iloc[0] - sol2_data['SolutionCost'].iloc[-1]:,}")

better_sol = "Solution 1" if sol1_data['SolutionCost'].iloc[-1] < sol2_data['SolutionCost'].iloc[-1] else "Solution 2"
print(f"\nBetter performing solution: {better_sol}")

print(f"\nAlgorithm characteristics:")
print(f"  - Uses dual solution approach (two parallel searches)")
print(f"  - High activity: generated {len(sol1_data) + len(sol2_data):,} total moves")
print(f"  - Significant cost improvements achieved")
print(f"  - Both solutions converged to similar cost ranges")
